- title: Fluent Python
- date: 2020-01-03 19:16
- category: study
- tags: study
- slug: fluent-python
- authors: Robin Rheem
- summary: Study notes for the book `Fluent Python`

# Fluent Python
The book has six parts.

1. Prologue
2. Data Structures
3. Functions as Objects
4. Object-Oriented Idioms
5. Control Flow
6. Metaprogramming

Data structures of what Python gives is something that I have to put in my head all the time. I always need to think of the type of data that I'm manipulating with. The space and time problem is always upon us. Picking the right data structure is one of the most important things to solve the time and space problem.

I think functions as objects will teach me functional programming. Passing functions around, doing something with those functions will actually be a nice foundation of functional programming. 

Object-Oriented idioms will teach me OOP in Python I guess.

Control flow? I think this part it'll give me tips and tricks on how to make cleaner code with control flow.

Metaprogramming is the thing that I'm really excited about. Code making code! Hope I get the hand of this and get max productivity. Hope I could make a framework out of it!

# The Python Data Model
Think of it as a framework. If you implement some special methods, that data model will be able to use some special Python syntax. Like accessing an array type of syntax if you implement the `__get_item__` method. By following this rule, you can implement, support, and interact with basic language constructs such as:

- Iteration
- Collections
- Attribute access
- Operator overloading
- Function and method invocation
- Object creation and destruction
- String representation and formatting
- Managed contexts (i.e., `with` blocks)

Magic and Dunder: Magic method == slang for special method. Specific special method, Dunder(double underscore) {method_name}.


## A Pythonic Card Deck
Demonstrating the power of implementing `__getitem__` and `__len__`.

In [0]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [3]:
deck = FrenchDeck()
len(deck)  # __len__ implementation!

52

In [5]:
deck[0]
deck[-1]  # __getitem__ makes this possible!

Card(rank='A', suit='hearts')

In [6]:
from random import choice

choice(deck)  # Random choice! If the __getitem__ is implemented, it can do this.

Card(rank='3', suit='spades')

Let's see the advantages of this approach.
* The users of your classes don't have to memorize arbitrary method names for standard operations("How to get the number of items? Is it `.size()`, `.length()`, or what?").
* It's easier to benefit from the rich Python standard library and avoid reinventing the wheel, like the `random.choice` function. 

Hmm ... interesting. Leveraging the standard library is something that I would really love to do. I never really thought of using the fundamental language constructs, and I always tend to just use libraries. When I saw gunicorn, it was really a new world open becaues it didn't even use anything. They just implemented that high-performant WSGI web server with just the Python standard library. I want to be like that too!

In [7]:
deck[:3]  # Automatically supports slicing because of __getitem__!

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [0]:
for card in deck:  # doctest: +ELLIPSIS
    print(card)  # Can't believe that __getitem__ makes it iterable.

In [0]:
for card in reversed(deck):  # doctest: +ELLIPSIS
    print(card)  # Didn't know you could reverse it this easily!

ELLIPSIS in Doctests.

This will let us be able to pass the doctest with (...) if there're a lot of things to print.

In [10]:
Card('Q', 'hearts') in deck  # The in keyword works because it's iterable!

True

In [0]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]


for card in sorted(deck, key=spades_high):
    print(card)  # Sorting!

Shuffling can be implemented with `__setitem__`, but that'll be at Chapter 11!

## How Special Methods Are Used
Special methods are supposed to be called by the interpretter. Also, if you're going to call them, it's best to call them when you're metaprogramming since it means that you're programming an interpreter of some kind. 
 Remember to call the build-in methods instead of calling them because they're faster.


### Emulating Numeric Types
In Python, the `+` operator can be overrided. I saw this in Scala, but couldn't see it in Kotlin. The ability to override operators is actually a great way to implement mathematical functions. No wonder Scala was sort of big in the big data era.

In [3]:
from math import hypot

class Vector:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        return f'Vector({self.x}, {self.y})'

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))
    
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)


# Addition!
v1 = Vector(2, 4)
v2 = Vector(2, 1)
print(v1 + v2)

# Absolute? For complex numbers
v = Vector(3, 4)
print(abs(v))

# Multiply with scalar value
print(v * 3)
# Print absolute
print(abs(v * 3))

Vector(4, 5)
5.0
Vector(9, 12)
15.0


### String Representation
The `__repr__` method actually gives a string representation of the object. If there's no representation, we'll be able to see the object address. Or maybe hash code or something. 
 It's best to look like the constructor. It'll be easy to reconstruct with that representation. Matching the source code is important! 
 `__repr__` is for developers and `__str__` is for customers.

### Arithmetic Operators
Create and return new objects than just change the properties. Don't mutate, just make it immutable. That's the important thing.

### Boolean Value of a Custom Type
The `__len__`, `__bool__`, things that go into `if`, `while`, `or`, `and`, `not`, determining whether it's truthy or falsey.

## Overview of Special Methods
The Python Language Reference has 83 method names and 43 of them implement arithmetic, bitwise, and comparison operators.

## Why len Is Not a Method
It's best to treat these special dunder methods as unary operators. They also operate in special ways too. Like calling the native CPython code for getting the length of a list.

## Chapter Summary
The biggest takeaway of this chapter is the Python data model. It defines what Pythonic means. Those special dunder methods leads us to call unary operators on the model and to have maximum practicality. Whatever we call len, will give the length of the model.

## Further Reading
* Data Model chapter of The Python Language Reference
* Python in a Nutshell, 2nd Edition
* Python Essential Reference, 4th Edition
* Python Cookbook, 3rd Edition
* The Art of the Metaobject Protocol

# An Array of Sequences
Didn't know texts, lists, and tables together were called trains `[...]`. Guido also already was doing a 10-year research project that was designing a programming environment for beginners. No wonder all the 'Pythonic' stuff came from! It was a language called ABC!

## Overview of Built-In Sequences